# ![uc-logo](./docs/images/favicon.ico) Automated Speeding Infringement Notice

---

In real world scenarios, some companies provide and manage fleets of vehicles.

Sometimes the company collects metrics for safety and insurance - such as speed monitoring!

We can simulate this real world process with our rovers and **IoT**!

![happy-rover](./docs/images/RVR_SDK_icons_RVR-Talking.png)

## Instructions

To test this code example and make monitor the rover's speed, follow these steps:

1. Click on the first code cell
1. press the play button to run the cell
1. Run each cell of code in order until the end
1. You can run the last cell many times


***You can follow along with the code by reading the comments! Example of comments are in the next cell...***

In [ ]:
""" This is a comment """

''' This is a comment '''

# this is a comment
# comments are lines in code that are not executed.
# They are for the programmer to help document their work.

### Setting Things Up

---

To use the IoT messaging service (message queuing telemetry transport - MQTT), and an email service in our program, we need to set some things up.

- **Run the following code cells to set everything up.**

In [ ]:
''' this cell installs some libraries to help with the rest of the code '''

# install some important python libraries
%pip install -r requirements.txt

In [ ]:
''' this code cell imports the libraries we installed to the current program '''

# import python libraries
import smtplib
import ssl
import paho.mqtt.client as mqtt
import time
from python.infringement_helper import InfringementHelper

In [ ]:
### DECLARE GLOBAL VARIABLES ###
# messaging topics
SPEED_TOPIC = "spheroRVR/speed"
LED_BLINK_TOPIC = "spheroRVR/led/blink"
LED_GREEN_TOPIC = "spheroRVR/led/green"

# The speed limit
MAXIMUM_SPEED_LIMIT = 0.4

### The Rover IP Address

---

Change the IP Address to match your rover.

1. Find the name printed on your rover - for example `robotics6`
1. The number of the rover is related to the ip address - the last set of digits is `100 + robot#`
1. For example, for `robotics5` the IP Address is `192.168.1.105` (100 + 5)

In [ ]:
### ROVER IP ADDRESS ###
###
###  CHANGE ME!
### 
###  for example: robotics7 has IP Address "192.168.1.107"
###
BROKER_HOSTNAME = "192.168.1.107"

In [ ]:
# set up minimal messaging service functions
def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("Connected to broker")

        global Connected  # Use global variable
        Connected = True  # Signal connection

    else:
        print(rc)
        print("Connection failed")


In [ ]:
# the email function
def send_email():
    # Set up your Gmail account details
    sender_email = 'engrehsanamiri@gmail.com'
    receiver_email = 'engrehsanamiri@gmail.com'
    password = 'jlaezogdlhjnllwu'
    subject = 'Test email'
    body = 'This is a test email sent from Python.'
    msg = f'Subject: {subject}\n\n{body}'

    # Set up the email message
    message = """\
    Subject: Test email from Python

    This is a test email sent from Python."""

    # Create a secure SSL context
    context = ssl.create_default_context()

    try:
        # Log in to your Gmail account and create a secure connection
        with smtplib.SMTP_SSL('smtp.gmail.com', 465, context=context) as server:
            server.login(sender_email, password)

            # Send the email message
            server.sendmail(sender_email, receiver_email, msg)

        print('Email sent successfully.')
    except Exception as e:
        print('Error sending email: %s', e)

In [ ]:
# create a tracking object to help track infringement events
infringement_tracker = InfringementHelper(MAXIMUM_SPEED_LIMIT)

### The Body of the Email

---

This is the body of the email.

It is a `string` of text that will be sent in an infringement notice.

- **You can edit the text to say something different!**

In [ ]:
# the email body string

email_body = ""

#### The message *'Callback'* function

---

This function is run when a message is received from the rover. This function:

1. reads the message
1. collects the speed data
1. tracks any speeding events
1. sends speeding infringement notices via email 

In [ ]:
# this function helps us listen to messages from the rover
# when we get a message about speed, we can check if it is too high
def on_message(client, userdata, message):
    speed = float(message.payload)

    if speed > 0.0:
        print("speed message received! -> " + str(speed))


    # check and track speeding events
    infringement_tracker.track_infringement(speed)
    
    if infringement_tracker.speeding_event_finished():
        notice_text = email_body
        infringement_tracker.send_infringement_notice(notice_text)

### The Main Function

---

This function is the *main* function. This function:

1. connects to the IoT messaging service
1. listens for messages about speed from the rover
1. sets up an automated response to speeding events - *( the callback function )*
1. if the rover's speed is too high, then an infringement will be sent by email

In [ ]:
def main():
    """ This function listens to rover messages and notifies of traffic infringements using mqtt and email
    """

    # Set up the MQTT client
    client = mqtt.Client("remote")
    client.on_connect = on_connect
    client.on_message = on_message
    
    # reset infringement tracker
    infringement_tracker.reset()

    # Connect to the MQTT broker
    client.connect(BROKER_HOSTNAME, 1883)
    client.loop_start()  # start the loop

    # subscribe to a message on a topic
    client.subscribe(SPEED_TOPIC)

    try:
        start_time = time.time()
        print("speeding detection program started...")
        while True:
            print("speeding detection program listening...")
            time.sleep(5)

            # exit after a little while (30 seconds or 1/2 minute)
            if time.time() - start_time > 30:
                print("speeding detection program has run for 30 seconds - exiting")
                break

    except KeyboardInterrupt:
        print("code interrupted")

    finally:
        print("exiting")
        client.disconnect()
        client.loop_stop()


### The Main Program

---

This code cell runs the main function. 

**You can run this cell many times**

In [ ]:
# test the main function and run the program
main()

In [ ]:
""" this cell is the end of the notebook, you do not have to run this cell """